In [64]:
DATASET_PATH = "TempDataset\normal\*.jpeg" # the dataset file or root folder path.
import os
import numpy as np
from PIL import Image
from os.path import basename, join

import glob
files = np.array([ f for f in glob.glob("TempDataset\cancer\*.jpeg")])
for file in files :
    f = basename(file)
    myimage = Image.open(file)
    #myimage = myimage.rotate(180)
    #r180 = "myrot_180_" + f ;
    #sf = "TempDataset\cancer\\"+ r180
    #myimage.save(sf)
    #myimage = np.fliplr(myimage)
    #myimage = Image.fromarray(myimage)
    #r180 = "fliplr_" + f ;
    #sf = "TempDataset\cancer\\"+ r180
    #myimage.save(sf)
    x_shift = 250
    y_shift = 150
    a = 1
    b = 0
    c = x_shift #left/right (i.e. 5/-5)
    d = 0
    e = 1
    g = y_shift #up/down (i.e. 5/-5)
    translate = myimage.transform(myimage.size, Image.AFFINE, (a, b, c, d, e, f))
    size = (translate.size[0] - x_shift, translate.size[1] - y_shift)
    translate = translate.transform(size, Image.EXTENT, (0, 0, size[0], size[1]))
    r180 = "xy250x150_" + f
    sf = "TempDataset\cancer\\"+ r180
    translate.save(sf)

In [18]:
read_images()

OSError: [WinError 123] The filename, directory name, or volume label syntax is incorrect: 'E:\\Projects\\HolisticClassification\\TempDataset\normal\\'

In [65]:

#taken from the several iciar 2018 implementations

import argparse
import numpy as np
import cv2
from scipy import ndimage
import os
from os.path import basename, join, exists
from os import makedirs
#from threaded_generator import threaded_generator
from time import time
import sys
np.random.seed(13)

PATCH_SIZES = [224]
SCALES = [0.5]

DEFAULT_INPUT_DIR = "data/train"
DEFAULT_PREPROCESSED_ROOT = "data/preprocessed/train"

PATCHES_PER_IMAGE = 20
AUGMENTATIONS_PER_IMAGE = 50
COLOR_LO = 0.7
COLOR_HI = 1.3
BATCH_SIZE = 16     # decrease if necessary

NUM_CACHED = 160


In [106]:
def normalize_staining(img):
   
    Io = 240
    beta = 0.15
    alpha = 1
    HERef = np.array([[0.5626, 0.2159],
                      [0.7201, 0.8012],
                      [0.4062, 0.5581]])
    maxCRef = np.array([1.9705, 1.0308])

    h, w, c = img.shape
    img = img.reshape(h * w, c)
    OD = -np.log((img.astype("uint16") + 1) / Io)
    ODhat = OD[(OD >= beta).all(axis=1)]
    if ODhat.shape[0] < 0 or ODhat.shape[1]<0 :
        return
    W, V = np.linalg.eig(np.cov(ODhat, rowvar=False))

    Vec = -V.T[:2][::-1].T  
    That = np.dot(ODhat, Vec)
    phi = np.arctan2(That[:, 1], That[:, 0])
    minPhi = np.percentile(phi, alpha)
    maxPhi = np.percentile(phi, 100 - alpha)
    vMin = np.dot(Vec, np.array([np.cos(minPhi), np.sin(minPhi)]))
    vMax = np.dot(Vec, np.array([np.cos(maxPhi), np.sin(maxPhi)]))
    if vMin[0] > vMax[0]:
        HE = np.array([vMin, vMax])
    else:
        HE = np.array([vMax, vMin])

    HE = HE.T
    Y = OD.reshape(h * w, c).T

    C = np.linalg.lstsq(HE, Y)
    maxC = np.percentile(C[0], 99, axis=1)

    C = C[0] / maxC[:, None]
    C = C * maxCRef[:, None]
    Inorm = Io * np.exp(-np.dot(HERef, C))
    Inorm = Inorm.T.reshape(h, w, c).clip(0, 255).astype("uint8")

    return Inorm


def hematoxylin_eosin_aug(img, low=0.7, high=1.3, seed=None):
    D = np.array([[1.88, -0.07, -0.60],
                  [-1.02, 1.13, -0.48],
                  [-0.55, -0.13, 1.57]])
    M = np.array([[0.65, 0.70, 0.29],
                  [0.07, 0.99, 0.11],
                  [0.27, 0.57, 0.78]])
    Io = 240

    h, w, c = img.shape
    OD = -np.log10((img.astype("uint16") + 1) / Io)
    C = np.dot(D, OD.reshape(h * w, c).T).T
    r = np.ones(3)
    r[:2] = np.random.RandomState(seed).uniform(low=low, high=high, size=2)
    img_aug = np.dot(C, M) * r

    img_aug = Io * np.exp(-img_aug * np.log(10)) - 1
    img_aug = img_aug.reshape(h, w, c).clip(0, 255).astype("uint8")
    return img_aug

def zoom_aug(img, zoom_var, seed=None):
    scale = np.random.RandomState(seed).uniform(low=1 / zoom_var, high=zoom_var)
    resized_img = cv2.resize(img, None, fx=scale, fy=scale, interpolation=cv2.INTER_CUBIC)
    return resized_img

def get_crops(img, size, n, seed=None):
    h, w, c = img.shape
    assert all([size < h, size < w])
    crops = []
    for _ in range(n):
        top = np.random.randint(low=0, high=h - size + 1)
        left = np.random.randint(low=0, high=w - size + 1)
        crop = img[top: top + size, left: left + size].copy()
        crop = np.rot90(crop, np.random.randint(low=0, high=4))
        if np.random.random() > 0.5:
            crop = np.flipud(crop)
        if np.random.random() > 0.5:
            crop = np.fliplr(crop)
        crops.append(crop)

    crops = np.stack(crops)
    assert crops.shape == (n, size, size, c)
    return crops


def get_crops_free(img, size, n, seed=None):
    h, w, c = img.shape
    assert all([size < h, size < w])
    d = int(np.ceil(size / np.sqrt(2)))
    crops = []
    for _ in range(n):
        center_y = np.random.randint(low=0, high=h - size + 1) + size // 2
        center_x = np.random.randint(low=0, high=w - size + 1) + size // 2
        m = min(center_y, center_x, h - center_y, w - center_x)
        if m < d:
            max_angle = np.pi / 4 - np.arccos(m / d)
            top = center_y - m
            left = center_x - m
            precrop = img[top: top + 2 * m, left: left + 2 * m]
        else:
            max_angle = np.pi / 4
            top = center_y - d
            left = center_x - d
            precrop = img[top: top + 2 * d, left: left + 2 * d]

        precrop = np.rot90(precrop, np.random.randint(low=0, high=4))
        angle = np.random.uniform(low=-max_angle, high=max_angle)
        precrop = ndimage.rotate(precrop, angle * 180 / np.pi, reshape=False)

        precrop_h, precrop_w, _ = precrop.shape
        top = (precrop_h - size) // 2
        left = (precrop_w - size) // 2
        crop = precrop[top: top + size, left: left + size]

        if np.random.random() > 0.5:
            crop = np.flipud(crop)
        if np.random.random() > 0.5:
            crop = np.fliplr(crop)
        crops.append(crop)

    crops = np.stack(crops)
    assert crops.shape == (n, size, size, c)
    return crops


def norm_pool(features, p=3):
    return np.power(np.power(features, p).mean(axis=0), 1/p)


def encode(crops, model):
    features = model.predict(crops)
    pooled_features = norm_pool(features)
    return pooled_features


def process_image(image_file):
    img = cv2.imread(image_file)
    if SCALE != 1:
        img = cv2.resize(img, None, fx=SCALE, fy=SCALE, interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_norm = normalize_staining(img)
    if(img_norm == NULL)
    for _ in range(AUGMENTATIONS_PER_IMAGE):
        img_aug = hematoxylin_eosin_aug(img_norm, low=COLOR_LO, high=COLOR_HI)
        single_image_crops = get_crops(img_aug, PATCH_SZ, PATCHES_PER_IMAGE)
        yield single_image_crops


def crops_gen(file_list):
    for i, (image_file, output_file) in enumerate(file_list):
        print("Crops generator:", i + 1,image_file,output_file)
        for crops in process_image(image_file):
            yield crops, output_file


def features_gen(crops_and_output_file, model):
    #print("features gen");
    ts = time()
    current_file = None
    pooled_features = []
    i = 0
    for j, (crops, output_file) in enumerate(crops_and_output_file):
        #print("cropping",j)
        #print(crops, output_file)
        if current_file is None:
            current_file = output_file
        features = encode(crops, model)
        if output_file == current_file:
            pooled_features.append(features)
        else:
            np.save(current_file, np.stack(pooled_features))
            pooled_features = [features]
            current_file = output_file
            average_time = int((time() - ts) / (i + 1))
            print("Feature generator: {}, {} sec/image.".format(i + 1, average_time))
            i += 1
    if len(pooled_features) > 0:
        np.save(current_file, np.stack(pooled_features))





In [78]:
#read_images_convert_to_jpeg()
files = np.array([ f for f in glob.glob("TempDataset\cancer\*.jpeg")])
for file in files :
    f = basename(file)
    myimage = cv2.imread(file)
    myimage = cv2.cvtColor(myimage, cv2.COLOR_BGR2RGB)
    img_aug = hematoxylin_eosin_aug(myimage, low=0.1, high=0.3)
    img_aug = Image.fromarray(img_aug)
    r180 = "img_aug0.1x0.3" + f
    sf = "TempDataset\cancer\\"+ r180
    img_aug.save(sf)
    

KeyboardInterrupt: 

In [85]:
files = np.array([basename(f) for f in glob.glob("TempDataset\cancernormal\\")])
i = 2435;
for filename in os.listdir("TempDataset\cancernormal\\"): 
        dst ="normal_" + str(i)+".jpeg"
        src ="TempDataset\cancernormal\\"+ filename 
        dst ="TempDataset\Train\\"+ dst 
        i+=1  
        # rename() function will 
        # rename all the files 
        os.rename(src, dst) 
print(i)

3236


In [86]:
import numpy as np

import tensorflow as tf
class ResNet:
    __name__ = "ResNet"

    def __init__(self, batch_size=16):
        self.model = tf.contrib.keras.applications.resnet50.ResNet50(include_top=True, weights='imagenet', pooling="avg")
        self.batch_size = batch_size
        self.data_format =  tf.keras.backend.image_data_format()

    def predict(self, x):
        if self.data_format == "channels_first":
            x = x.transpose(0, 3, 1, 2)
        x = tf.contrib.keras.applications.resnet50.preprocess_input(x.astype(tf.keras.backend.floatx()))
        return self.model.predict(x, batch_size=16)

In [108]:
import os
from os.path import basename, join, exists
DATASET_PATH = 'TempDataset\Train\\' # the dataset file or root folder path.


NUM_CACHED = 160
NN_MODELS = [ResNet]
for SCALE in SCALES:
        print("SCALE:", SCALE)
        for NN_MODEL in NN_MODELS:
            print("NN_MODEL:", NN_MODEL.__name__)
            for PATCH_SZ in PATCH_SIZES:
                print("PATCH_SZ:", PATCH_SZ)
                model = NN_MODEL(batch_size=BATCH_SIZE)
                files = np.array([f for f in glob.glob(DATASET_PATH+"*.jpeg")])

                output_files =[f.replace("jpeg", "npy") for f in files]
                
                file_list = zip(files, output_files)
                crops_and_output_file = crops_gen(file_list)
                #crops_and_output_file_ = threaded_generator(crops_and_output_file, num_cached=NUM_CACHED)
                features_gen(crops_and_output_file, model)

SCALE: 0.5
NN_MODEL: ResNet
PATCH_SZ: 224
Crops generator: 1 TempDataset\Train\normal_2549.jpeg TempDataset\Train\normal_2549.npy


E:\Anaconda3\envs\tfdeeplearning\lib\site-packages\ipykernel_launcher.py:34: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


Crops generator: 2 TempDataset\Train\normal_2550.jpeg TempDataset\Train\normal_2550.npy
Feature generator: 1, 387 sec/image.
Crops generator: 3 TempDataset\Train\normal_2551.jpeg TempDataset\Train\normal_2551.npy
Feature generator: 2, 385 sec/image.
Crops generator: 4 TempDataset\Train\normal_2552.jpeg TempDataset\Train\normal_2552.npy
Feature generator: 3, 384 sec/image.
Crops generator: 5 TempDataset\Train\normal_2553.jpeg TempDataset\Train\normal_2553.npy


C:\Users\naveen\AppData\Roaming\Python\Python35\site-packages\numpy\lib\function_base.py:356: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
C:\Users\naveen\AppData\Roaming\Python\Python35\site-packages\numpy\core\_methods.py:78: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
E:\Anaconda3\envs\tfdeeplearning\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: Degrees of freedom <= 0 for slice
C:\Users\naveen\AppData\Roaming\Python\Python35\site-packages\numpy\lib\function_base.py:2326: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
C:\Users\naveen\AppData\Roaming\Python\Python35\site-packages\numpy\lib\function_base.py:2326: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


LinAlgError: Array must not contain infs or NaNs